Table of contents:

  Loading and preProcessing data.
  
  Extracting text based features.

    1. Special char and numbers.
    2. Word count.
    3. Number of chars.
    4. Avg word length.
    5. Stop Words.
    6. POS tags.
    7. NER.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re

In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp=spacy.load('en_core_web_sm')

In [7]:
df=pd.read_csv('/content/drive/My Drive/tweets.csv',nrows=1000)

In [8]:
df

,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted
0,RT @rssurjewala: Critical question: Was PayTM ...,False,0,NaN,2016-11-23 18:40:30,False,NaN,8.014957e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",HASHTAGFARZIWAL,331,True,False
1,RT @Hemant_80: Did you vote on #Demonetization...,False,0,NaN,2016-11-23 18:40:29,False,NaN,8.014957e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",PRAMODKAUSHIK9,66,True,False
2,"RT @roshankar: Former FinSec, RBI Dy Governor,...",False,0,NaN,2016-11-23 18:40:03,False,NaN,8.014955e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",rahulja13034944,12,True,False
3,RT @ANI_news: Gurugram (Haryana): Post office ...,False,0,NaN,2016-11-23 18:39:59,False,NaN,8.014955e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",deeptiyvd,338,True,False
4,RT @satishacharya: Reddy Wedding! @mail_today ...,False,0,NaN,2016-11-23 18:39:39,False,NaN,8.014954e+17,NaN,"<a href=""http://cpimharyana.com"" rel=""nofollow...",CPIMBadli,120,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,#Demonetization wrestling... <ed><U+00A0><U+00...,False,4,NaN,2016-11-23 09:02:55,False,NaN,8.013503e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",shekhar_ojha,3,False,False
996,Will CBI raid now on AB . AR .. Panama .. ? #D...,False,0,NaN,2016-11-23 09:00:43,False,NaN,8.013497e+17,NaN,"<a href=""http://twitter.com/download/iphone"" r...",muntaziraapka,3,False,False
997,RT @Blunt_Tinku14: @DineshR1511 they're under ...,False,0,NaN,2016-11-23 09:00:05,False,NaN,8.013496e+17,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",veritas47vincit,2,True,False
998,Can't believe this is happening in app store. ...,False,0,NaN,2016-11-23 08:59:51,False,NaN,8.013495e+17,NaN,"<a href=""http://twitter.com/download/iphone"" r...",bhaveshgwalani,0,False,False


In [9]:
df.head(2)

,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted
0,RT @rssurjewala: Critical question: Was PayTM ...,False,0,NaN,2016-11-23 18:40:30,False,NaN,8.014957e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",HASHTAGFARZIWAL,331,True,False
1,RT @Hemant_80: Did you vote on #Demonetization...,False,0,NaN,2016-11-23 18:40:29,False,NaN,8.014957e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",PRAMODKAUSHIK9,66,True,False


In [14]:
df.drop(df.columns[1:],axis=1,inplace=True)

In [15]:
df

,text
0,RT @rssurjewala: Critical question: Was PayTM ...
1,RT @Hemant_80: Did you vote on #Demonetization...
2,"RT @roshankar: Former FinSec, RBI Dy Governor,..."
3,RT @ANI_news: Gurugram (Haryana): Post office ...
4,RT @satishacharya: Reddy Wedding! @mail_today ...
...,...
995,#Demonetization wrestling... <ed><U+00A0><U+00...
996,Will CBI raid now on AB . AR .. Panama .. ? #D...
997,RT @Blunt_Tinku14: @DineshR1511 they're under ...
998,Can't believe this is happening in app store. ...


No charts were generated by quickchart


In [17]:
df.loc[0,'text']

"RT @rssurjewala: Critical question: Was PayTM informed about #Demonetization edict by PM? It's clearly fishy and requires full disclosure &amp;\x85"

In [18]:
df.loc[512,'text']

'RT @smita_muk: BREAKING NEWS\r\nPMapps result amnounced!\r\n90% Indians support #demonetization\r\n<ed><U+00A0><U+00BD><ed><U+00B1><U+008F><ed><U+00A0><U+00BD><ed><U+00B1><U+008F><ed><U+00A0><U+00BD><ed><U+00B1><U+008F><ed><U+00A0><U+00BD><ed><U+00B1><U+008F><ed><U+00A0><U+00BD><ed><U+00B1><U+008F><ed><U+00A0><U+00BD><ed><U+00B1><U+008F><U+270C><U+270C><U+270C><U+270C><U+270C><U+270C><ed><U+00A0><U+00BD><ed><U+00B1><U+0086><ed><U+00A0><U+00BD><ed><U+00B1><U+0086><ed><U+00A0><U+00BD><ed><U+00B1><U+0086><ed><U+00A0><U+00BD><ed><U+00B1><U+0086><ed><U+00A0><U+00BD><ed><U+00B1><U+0086><ed><U+00A0><U+00BD><ed><U+00B1><U+0086>\r\n@narendramodi Zindabad!'

In [19]:
# preprocess tweets
def preprocess(text):
  # remove unicode characters
  text=re.sub(r"<U\+[A-Z0-9]+>|<ed>","",text)
  # remove newline and rawstring characters
  text=re.sub(r"\n|\r","",text)
  return text

In [20]:
# Apply function
df['text']=df['text'].apply(preprocess)

In [21]:
df.head()

,text
0,RT @rssurjewala: Critical question: Was PayTM ...
1,RT @Hemant_80: Did you vote on #Demonetization...
2,"RT @roshankar: Former FinSec, RBI Dy Governor,..."
3,RT @ANI_news: Gurugram (Haryana): Post office ...
4,RT @satishacharya: Reddy Wedding! @mail_today ...


In [22]:
df.shape

(1000, 1)

Extracting text based features

In [32]:
#no of mentions used in tweets
def mention(text):
  mention_list=re.findall('@\w+',text)
  return len(mention_list)

In [33]:
df['mentions_count']=df['text'].apply(mention)

In [34]:
df['mentions_count']

0      1
1      1
2      1
3      1
4      2
      ..
995    0
996    0
997    2
998    0
999    0
Name: mentions_count, Length: 1000, dtype: int64

In [35]:
df['mentions_count'].describe()

count    1000.000000
mean        0.866000
std         0.988444
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         8.000000
Name: mentions_count, dtype: float64

In [36]:
def hashtags(text):
  hash=re.findall('#\w+',text)
  return len(hash)

In [37]:
df['hashtags_count']=df['text'].apply(hashtags)

In [38]:
df['hashtags_count'].head()

0    1
1    1
2    1
3    1
4    2
Name: hashtags_count, dtype: int64

In [39]:
df['hashtags_count'].describe()

count    1000.000000
mean        1.688000
std         1.272114
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        10.000000
Name: hashtags_count, dtype: float64

In [40]:
#Number of name titles in tweet
def title(text):
  count=re.findall('Mr\.|Mrs\.|Dr\.|Miss\s*',text)
  return len(count)

In [41]:
df['text'].apply(title)

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: text, Length: 1000, dtype: int64

In [42]:
# Word count
# list comprehension to count number of words in tweet
df['word_count']=[len(i.split()) for i in df['text']]

In [43]:
df.head(5)

,text,mentions_count,hashtags_count,word_count
0,RT @rssurjewala: Critical question: Was PayTM ...,1,1,20
1,RT @Hemant_80: Did you vote on #Demonetization...,1,1,11
2,"RT @roshankar: Former FinSec, RBI Dy Governor,...",1,1,21
3,RT @ANI_news: Gurugram (Haryana): Post office ...,1,1,16
4,RT @satishacharya: Reddy Wedding! @mail_today ...,2,2,9


In [44]:
df['word_count'].describe()

count    1000.000000
mean       16.685000
std         4.566468
min         3.000000
25%        14.000000
50%        17.000000
75%        20.000000
max        28.000000
Name: word_count, dtype: float64

In [45]:
#No of characters
#list comprehension to cunt number of chars
df['character_count']=[len(i) for i in df['text']]

In [46]:
df.head()

,text,mentions_count,hashtags_count,word_count,character_count
0,RT @rssurjewala: Critical question: Was PayTM ...,1,1,20,144
1,RT @Hemant_80: Did you vote on #Demonetization...,1,1,11,66
2,"RT @roshankar: Former FinSec, RBI Dy Governor,...",1,1,21,138
3,RT @ANI_news: Gurugram (Haryana): Post office ...,1,1,16,140
4,RT @satishacharya: Reddy Wedding! @mail_today ...,2,2,9,107


In [47]:
df['character_count'].describe()

count    1000.000000
mean      124.673000
std        21.427861
min        34.000000
25%       117.000000
50%       135.000000
75%       139.000000
max       148.000000
Name: character_count, dtype: float64

Average word length

In [50]:
def avg_word_len(text):
  # variable to store word lengths
  word_lens=0
  # Iterate over all the words in tweet
  for token in text.split():
    word_lens+=len(token)
  # no of words in tweet
  word_count=text.split()
  # Return average length of words in tweet
  return word_lens/len(word_count)

In [51]:
df['avg_word_len']=df['text'].apply(avg_word_len)

In [52]:
df.head(5)

,text,mentions_count,hashtags_count,word_count,character_count,avg_word_len
0,RT @rssurjewala: Critical question: Was PayTM ...,1,1,20,144,6.200000
1,RT @Hemant_80: Did you vote on #Demonetization...,1,1,11,66,5.090909
2,"RT @roshankar: Former FinSec, RBI Dy Governor,...",1,1,21,138,5.571429
3,RT @ANI_news: Gurugram (Haryana): Post office ...,1,1,16,140,7.750000
4,RT @satishacharya: Reddy Wedding! @mail_today ...,2,2,9,107,11.000000


In [53]:
df['avg_word_len'].describe()

count    1000.000000
mean        6.865341
std         1.698611
min         3.892857
25%         5.650000
50%         6.578947
75%         7.648810
max        16.666667
Name: avg_word_len, dtype: float64

In [55]:
#stopwords
def stopwords(text):
  doc=nlp(text)
  count=0
  for token in doc:
    if token.is_stop==True:
      count+=1
  return count

In [56]:
df['stopwords']=df['text'].apply(stopwords)

In [58]:
df['stopwords'].head()

0    7
1    4
2    5
3    2
4    0
Name: stopwords, dtype: int64

In [57]:
df['stopwords'].describe()

count    1000.000000
mean        6.038000
std         3.285019
min         0.000000
25%         4.000000
50%         6.000000
75%         8.000000
max        19.000000
Name: stopwords, dtype: float64

In [59]:
#POS Tags
def pos(text):
  doc=nlp(text)
  count=0
  for token in doc:
    if token.pos_ in ["NOUN","ADP","ADJ"]:
      count+=1
  return count

In [60]:
df['pos']=df['text'].apply(pos)

In [61]:
df.head()

,text,mentions_count,hashtags_count,word_count,character_count,avg_word_len,stopwords,pos
0,RT @rssurjewala: Critical question: Was PayTM ...,1,1,20,144,6.200000,7,9
1,RT @Hemant_80: Did you vote on #Demonetization...,1,1,11,66,5.090909,4,4
2,"RT @roshankar: Former FinSec, RBI Dy Governor,...",1,1,21,138,5.571429,5,6
3,RT @ANI_news: Gurugram (Haryana): Post office ...,1,1,16,140,7.750000,2,10
4,RT @satishacharya: Reddy Wedding! @mail_today ...,2,2,9,107,11.000000,0,6


In [62]:
df.describe()

,mentions_count,hashtags_count,word_count,character_count,avg_word_len,stopwords,pos
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.866000,1.688000,16.685000,124.673000,6.865341,6.038000,7.657000
std,0.988444,1.272114,4.566468,21.427861,1.698611,3.285019,3.096601
min,0.000000,0.000000,3.000000,34.000000,3.892857,0.000000,0.000000
25%,0.000000,1.000000,14.000000,117.000000,5.650000,4.000000,5.000000
50%,1.000000,1.000000,17.000000,135.000000,6.578947,6.000000,8.000000
75%,1.000000,2.000000,20.000000,139.000000,7.648810,8.000000,10.000000
max,8.000000,10.000000,28.000000,148.000000,16.666667,19.000000,19.000000


In [ ]:
# NER

In [63]:
def ner(text):
  doc=nlp(text)
  count=0
  for ent in doc.ents:
    if ent.label_:
      count+=1
  return count

In [64]:
df['ner']=df['text'].apply(ner)

In [65]:
df.head(5)

,text,mentions_count,hashtags_count,word_count,character_count,avg_word_len,stopwords,pos,ner
0,RT @rssurjewala: Critical question: Was PayTM ...,1,1,20,144,6.200000,7,9,3
1,RT @Hemant_80: Did you vote on #Demonetization...,1,1,11,66,5.090909,4,4,1
2,"RT @roshankar: Former FinSec, RBI Dy Governor,...",1,1,21,138,5.571429,5,6,4
3,RT @ANI_news: Gurugram (Haryana): Post office ...,1,1,16,140,7.750000,2,10,1
4,RT @satishacharya: Reddy Wedding! @mail_today ...,2,2,9,107,11.000000,0,6,2
